# NO LONELY COLORINGS: All solutions and orientations
In this file we have three sections:  
   1. **Main backtracking function:** Cell that includes the main backtracking function and the imports that will be needed in this notebook.  
   2. **Obtaining all solutions for all graphs in a file:** Cell that can be used to go through all the files saved in a g6 file, in order to search all the no lonely colorings of the graphs saved. Before using this cell make sure that the variable 'folderPath' is defined as needed.  
   3. **Going through all orientations of an specific graph:** Cell that can be used to check an specific graph. We can obtain all the no lonely colorings and, if wanted, we can go through all its orientations. To do so, we have to make sure to update the variable 'g6_string' with the graph6 format of the graph that we want to check. **IMPORTANT:** In addition, if we want to examine the orientations, we will have to save a file named 'orientations.d6' in the same folder containing this notebook.

# Main backtracking function

In [ ]:
import os
import time
from collections import defaultdict
import supportFunctions as sf

def checkColorings_backtrack(G, G_aux, G_coloredEdges, edge, color, colorIsOkay, solutions,
                             statesChecked, lenMonochromaticCycle):
    """
    Backtracking function that searchs a no lonely coloring of G.
    
    PARAMETERS:
        G:                     Graph that we want to color
        G_aux:                 Graph that contains the edges that are not colored with "color"
        G_coloredEdges:        Graph that contains the colored edges
        edge:                  Edge that is going to be colored next
        color:                 Color that we will use to color "edge"
        colorIsOkay:           Boolean that tells us that, once we color "edge", if the edges colored
                                 with "color" satisfy the needed conditions to be a no lonely color
        statesChecked:         Set that contains the colorings that we have already discarded.
        lenMonochromaticCycle: If we find a monochromatic cycle, it contains the length. If not it is 0.
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """    
    u, v, label = edge[0], edge[1], edge[2]
    
    # If the edge is uncolored
    if label == 0:
        # We color the edge and update all the graphs
        G.set_edge_label(u, v, color)
        G_aux.delete_edge(u, v)
        G_coloredEdges.add_edge(u, v, color)
        
        # If this was the last edge to paint and the coloring is okay, we add the solution
        if len(G_coloredEdges.edges()) == len(G.edges()) and colorIsOkay:
            if tuple(G_coloredEdges.edges()) not in solutions:
                solutions.add(tuple(G_coloredEdges.edges()))
                # UNCOMMENT TO SEE THE SOLUTIONS IN THE OUTPUT
                #print(f"Solution {len(solutions)}: {G_coloredEdges.edges()}")
            
            # So that we do not stop searching for solutions
            return False

        # DEFINING edges_missing --> WE WANT TO KNOW WHICH EDGES SHOULD BE COLORED NEXT
        # We search for lonely uncolored edges in cycles
        if len(G_coloredEdges.edges()) != len(G.edges()): # If G is fully colored, there is no need
            checkNonColoredCycles, uncoloredEdges = sf.searchOfLonelyEdgesByColor(G, 0)
        else: checkNonColoredCycles = True
        
        if not checkNonColoredCycles:
            # uncoloredEdges contains a lonely uncolored edge in a cycle
            edges_missing = uncoloredEdges
        else:
            # Else we search for a lonely edge with color "color"
            checkCyclesColor, uncoloredEdges = sf.searchOfLonelyEdgesByColor(G, color)
            
            if not checkCyclesColor:
                # uncoloredEdges contains the edges of the cycle that contains a lonely edge
                edges_missing = uncoloredEdges
            else:
                # If we have not found a lonely edge of color "color"...
                if colorIsOkay:
                    # We continue coloring with another color
                    if checkColorings_backtrack(G, G.copy(), G_coloredEdges, sf.findNextEdge(G), color + 1,
                                             False, solutions, statesChecked, lenMonochromaticCycle):
                        return True
                
                # If we have no candidates, we just try to color all the uncolored edges
                edges_missing = G_aux.edges()
        
        # WE EXPLORE THE CANDIDATES TO COLOR
        for edge_to_color in edges_missing:
            u_to_color, v_to_color, label_to_color = edge_to_color[0], edge_to_color[1], edge_to_color[2]

            if label_to_color == 0:                
                G_coloredEdges.add_edge(u_to_color, v_to_color, color)

                # If we have not discarded this state before
                if tuple(G_coloredEdges.edges()) not in statesChecked:
                    # If the vertices do not have three adjacent edges of color "color"
                    if sf.isDegreeColorGood(G_coloredEdges, edge_to_color, color):
                        lenMonochromaticCycle_currentEdge = sf.searchMonochromaticCyclesInEdge(G_coloredEdges,
                                                                                    edge_to_color, color)
                        
                        # If the monochromatic cycles have the same length
                        if (
                            lenMonochromaticCycle_currentEdge == 0
                            or lenMonochromaticCycle == 0
                            or lenMonochromaticCycle_currentEdge == lenMonochromaticCycle
                        ):
                            if lenMonochromaticCycle == 0:
                                # We have not found a previous monochromatic cycle. So, we save the new value
                                lenMonochromaticCycle = lenMonochromaticCycle_currentEdge
                            
                            # We update G_aux
                            G_aux.delete_edge(u_to_color, v_to_color)

                            # If G_aux is disconnected and G_colored edges has both endpoints
                            # of each edge of color "color" in different components
                            if (
                                not G_aux.is_connected()
                                and sf.areEndpointsInDifferentComponents(G_coloredEdges,
                                                                      G_aux.connected_components(), color)
                            ):
                                # The edges of color "color" satisfy the necessary conditions
                                # --> colorIsOkay = True
                                if checkColorings_backtrack(G, G_aux, G_coloredEdges, edge_to_color, color,
                                                         True, solutions, statesChecked, lenMonochromaticCycle):
                                    return True

                            # If not, we continue considering that the color is not okay
                            # --> colorIsOkay = False
                            if checkColorings_backtrack(G, G_aux, G_coloredEdges, edge_to_color, color,
                                                     False, solutions, statesChecked, lenMonochromaticCycle):
                                return True
                            
                            # We restore G_aux
                            G_aux.add_edge(u_to_color, v_to_color, 0)

                # We restore G_coloredEdges
                G_coloredEdges.delete_edge(u_to_color, v_to_color)
        
        # NONE OF THE edges_missing CAN BE COLORED WITH color
        # To save memory, we do not save the cases that are the only option after a previous state
        if len(edges_missing) != 1: statesChecked.add(tuple(G_coloredEdges.edges()))
        
        G.set_edge_label(u, v, 0)
        G_aux.add_edge(u, v, 0)
        G_coloredEdges.delete_edge(u, v)

    return False

# Obtaining all solutions for all graphs in a file

In [ ]:
def checkAllColorings(G):
    """
    Functiong that calls the backtracking funcion that goes through all the colorings in the
    search of all no lonely coloring solutions.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """
    solutions = set()
    checkColorings_backtrack(G.copy(), G.copy(), Graph(), G.edges()[0], 1, False, solutions,
                             set(), 0)    
    
    print(f"We have found {len(solutions)} solutions.")
    return len(solutions) > 0    # So that it returns True when finds at least a solutions and False if not


def isNoLonelyColor(G):
    """
    Main functions that checks if a given graph has a no lonely coloring.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if G is a no lonely coloring.
        False if the graph is not a no lonely coloring.
    """
    return checkAllColorings(G)


# WRITE THE FOLDER THAT YOU WANT TO USE
folderPath = './'

# We go through the files in the folder "folderPath"
for file in os.listdir(folderPath):
    if file.endswith('.g6') or file.endswith('.graph6'):
        filePath = os.path.join(folderPath, file)

        check = input(f"Do you want to open the file: {file}? (y/n)")
        
        if check == 'y':            
            start_global = time.time()            
            f = open(filePath, "r")
            lines = f.readlines()

            for i, line in enumerate(lines):
                start = time.time()
                g6_string = line

                try:
                    G = Graph(g6_string)
                except Exception as e:
                    print(f"Exception: {e}")
                    print(f"Graph: {g6_string}")
                    continue

                # We set the labels of the edges as 0
                for u, v, _ in G.edges():
                    G.set_edge_label(u, v, 0)

                if not isNoLonelyColor(G):
                    # If we have not found a no lonely coloring...
                    print(f"Graph {i} in {file}: FALSE")
                    print(g6_string)

                    # Draw the SageMath graph
                    G.show()

                end = time.time()
                print(f"{g6_string} checked in {end-start}s")

            end_global = time.time()
            print(f"{file} checked in {end_global-start_global}s")

# Going through all orientations of an specific graph

In [ ]:
def has_high_degree(D, vertex, threshold=2):
    """
    Function that checks the incoming and outgoing degree of a vertex by color.
    
    PARAMETERS:
        D:         DiGraph
        vertex:    Vertex that is being checked
        threshold: Maximum degree permited
        
    RETURNS:
        True if the degree is greater or equal to the threshold.
        False if not.
    """
    counts = defaultdict(int)
    for _, __, label in D.incoming_edges(vertex):
        counts[label] += 1
        if counts[label] >= threshold:
            return True
    
    counts = defaultdict(int)
    for _, __, label in D.outgoing_edges(vertex):
        counts[label] += 1
        if counts[label] >= threshold:
            return True
    
    return False


def colored_orientation(G, orientation):
    """
    Function to create and color a directed graph according to the coloring of its undirected graph.
    
    PARAMETERS:
        G: Undirected colored graph
        orientation: string in d6 format of an orientation of G
    
    RETURNS:
        D: Colored Digraph
    """
    # We create a digraph with multiples edges allowed
    D = DiGraph(orientation, multiedges=True)
    
    if len(G) != len(D): sys.exit("ERROR: The orientations do not match with the graph.")
    
    # We color the digraph with the same coloring that has G
    for u, v, label in G.edges():
        if D.has_edge(u, v):
            D.set_edge_label(u, v, label)
        if D.has_edge(v, u):
            D.set_edge_label(v, u, label)
    
    return D


def follow_path(D, startVertex, path, index, v):
    """
    Recursive function that follows a given path.
    
    PARAMETERS:
        D:           Colored DiGraph
        startVertex: Vertex where the path has started
        path:        List of colors that represent a path
        index:       Index of the path in which we are at the moment
        v:           Current vertex
    
    RETURNS:
        True if we find a cycle
        False if not
    """
    color = path[index]

    if color > 0:  # If the color is positive is because we are using an outgoing edge
        for _, u, label in D.outgoing_edges(v):
            if label == color:
                if index == len(path) - 1:
                    # The path has ended
                    if u == startVertex: return True
                    else: return False
                else:
                    # We follow the path with the next vertex
                    return follow_path(D, startVertex, path, index+1, u)
    
    else:  # If the color is negative is because we are using an incoming edge
        for u, _, label in D.incoming_edges(v):
            if label == - color:
                if index == len(path) - 1:
                    # The path has ended
                    if u == startVertex: return True
                    else: return False
                else:
                    # We follow the path with the next vertex
                    return follow_path(D, startVertex, path, index+1, u)
    
    # We can't tell it's False because we have not been able to follow the entire path
    # (since we are searching for subgraphs, it could be that the path is correct in the hole Cayley graph)
    return True


def checkCyclePaths(D, G):
    """
    Function that verifies if the same color sequence, starting at different vertices, gives an isomorphic path.
    
    PARAMETERS:
        D: Colored DiGraph
        G: Colored Graph (undirected graph of D)
    
    RETURNS:
        True if the paths are isomorphic.
        False if we find two paths with the same color sequence that are not isomorphic.
    """
    # We obtain all possible cycles in G (also the ones that use an arc in the opposited direction in D)
    for cycle in G.to_directed().all_simple_cycles():  # Cycle contains the vertex of the origin twice
        if len(cycle) > 3:  # We skip cycles of length 2
            labelsInCycle = set()
            path = []
            for i in range(len(cycle) - 1):
                if D.has_edge(cycle[i], cycle[i+1]):
                    label = D.edge_label(cycle[i], cycle[i+1])[0]
                else:
                    # The edge is in the opposite direction. So, we save the label as negative
                    label = - D.edge_label(cycle[i+1], cycle[i])[0]

                path.append(label)
                labelsInCycle.add(label)
            
            # For every cyclic rotation of the path
            for i in range(len(path)):
                aux_path = path[i:] + path[:i]
                
                # For every vertex
                for v in D.vertices():
                    if not follow_path(D, v, aux_path, 0, v):
                        return False
        
    return True


def checkDirectedCycles(D):
    """
    Function that checks the lengths of the monochromatic cycles if there is a monochromatic cycle of length 2.
    We only check this if there is a cycle of length 2, because the length of the monochromatic cycle
    have already been checked with the undirected graph.
    
    PARAMETERS:
        D: Colored DiGraph
    
    RETURNS:
        True if the cycle lengths coincide.
        Flase if not.
    """
    labels_with_len2_cycles = set()
    cycles = D.all_simple_cycles()
    bigCycles = []
    label = None
    
    # We search for length 2 cycles
    len2_cycle_check = False
    for cycle in cycles:
        if len(cycle) == 3:
            label = D.edge_label(cycle[0], cycle[1])[0]
            len2_cycle_check = True
        else:
            # We save the big cycles, for afterwards
            bigCycles.append(cycle)
    
    # If we have a len 2 cycle, we check the rest of the cycles
    if len2_cycle_check:
        for cycle in bigCycles:
            # We search the monochromatic cycles
            colorsInCycle = set()
            monochromaticCheck = True
            for i in range(len(cycle) - 1):
                label = D.edge_label(cycle[i], cycle[i+1])[0]
                colorsInCycle.add(label)

                if len(colorsInCycle) != 1:   # We stop checking this cycle because it is not monochromatic
                    monochromaticCheck = False
                    break

            if monochromaticCheck:
                # We have found two cycles of the same color that have different lengths
                return False
        
    return True
        
        
def discardOrientations(G, filePath):
    """
    Function that goes through all the orientations saved in the file "filePath".
    
    PARAMETERS:
        G:        Colored Graph
        filePath: File that contains all the orientations in format d6
    """
    start_global = time.time()
    f = open(filePath, "r")
    lines = f.readlines()

    for i, line in enumerate(lines):
        d6_string = line[1:]

        try:
            D = colored_orientation(G, d6_string)
        except Exception as e:
            print(f"Exception: {e}")
            print(f"Graph: {d6_string}")
            continue
        
        # We check the degree
        check = True
        for v in D.vertices():
            if has_high_degree(D, v):
                check = False
                break

        if check:
            # We check the length of the cycles
            check = checkDirectedCycles(D)

            if check:
                # We check the paths
                if checkCyclePaths(D, G):
                    D.show()
                    print(d6_string)
                    # RECOMMENDED TO UNCOMMENT FOR LARGE GRAPHS
                    #sys.exit("Program interrupted: CORRECT COLORING FOUND.")

    end_global = time.time()
    print(f"Checked in {end_global-start_global}s")


def checkAllColorings(G):
    """
    Functiong that calls the backtracking funcion that goes through all the colorings in the
    search of all no lonely coloring solutions. Also, if wanted, we go through all the possible
    orientations of the graph.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """
    solutions = set()
    checkColorings_backtrack(G.copy(), G.copy(), Graph(), G.edges()[0], 1, False, solutions,
                             set(), 0)

    print(f"We have found {len(solutions)} solutions")
    check = input("Do you want to go through all the colorings of the graph? REMEMBER TO UPDATE orientations.d6 BEFORE (y/n)")
    
    if check == 'y':
        # We start checking the orientations for every solution
        for i, solution in enumerate(solutions):
            print(f"Solution {i+1}: {solution}")
            discardOrientations(Graph(list(solution)), "orientations.d6")
    
    return len(solutions) > 0  # So that it returns True when finds at least a solutions and False if not


def isNoLonelyColor(G):
    """
    Main functions that checks if a given graph has a no lonely coloring.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if G is a no lonely coloring.
        False if the graph is not a no lonely coloring.
    """
    return checkAllColorings(G)


# WRITE THE GRAPH IN FORMAT g6
g6_string = r"HCQf@o["

try:
    G = Graph(g6_string)
except:
    print("ERROR: Introduce a valid graph in g6 format.")
    G = Graph()

# We set the labels of the edges as 0
for u, v, _ in G.edges():
    G.set_edge_label(u, v, 0)

start = time.time()
if not isNoLonelyColor(G):
    print("No lonely color graph found.")
    
    # Draw the SageMath graph
    G.show()

end = time.time()
print(f"{g6_string} checked in {end-start}s")